In [6]:
# pip install ctc-segmentation
# pip install speechbrain

from speechbrain.pretrained import EncoderDecoderASR
from speechbrain.alignment.ctc_segmentation import CTCSegmentation

# Requires a model with CTC output
asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-transformer-transformerlm-librispeech")
aligner = CTCSegmentation(asr_model, kaldi_style_text=False)


In [7]:
audio_path = "/vol/tensusers2/wharmsen/SERDA/round1/audio/stories/2RRDV-story_1-20230113142045040.wav"

text = ["bang in het donker als de"]
segments = aligner(audio_path, text, name="example1")
print(segments)
# example1_0000 example1 0.04 0.70 -0.0122 THE BIRCH CANOE
# example1_0001 example1 0.97 1.97 -0.0295 SLID ON THE
# example1_0002 example1 1.97 3.00 -0.0258 SMOOTH PLANKS

RuntimeError: The size of tensor a (4604) must match the size of tensor b (2500) at non-singleton dimension 1

In [2]:
import librosa
y, sr = librosa.load(audio_path, sr=16000)
y_segment = y[0:10]

text = ["bang in het donker als de"]
segments = aligner(y_segment, text, name="example1")
print(segments)

NameError: name 'aligner' is not defined

In [3]:
import torch
import numpy as np
from typing import List
import ctc_segmentation
from datasets import load_dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer

cache_dir = '/vol/tensusers5/wharmsen/wav2vec2/cache/'

# load model, processor and tokenizer
model_name = "FremyCompany/xls-r-2b-nl-v2_lm-5gram-os2_hunspell" #"jonatasgrosman/wav2vec2-large-xlsr-53-dutch"
processor = Wav2Vec2Processor.from_pretrained(model_name, cache_dir=cache_dir)
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = Wav2Vec2ForCTC.from_pretrained(model_name, cache_dir=cache_dir)

# load dummy dataset and read soundfiles
SAMPLERATE = 16000
audio_path = "/vol/tensusers2/wharmsen/SERDA/round1/audio/stories/2RRDV-story_1-20230113142045040.wav"
# ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
audio, sr = librosa.load(audio_path, sr=16000)

prompt = "bang in het donker als de beker vol is doet marte de kraan dicht nu moet je echt gaan slapen zegt mama je kwam al drie keer uit bed ben je niet blij met je eigen slaapkamer marte voelt haar wangen warm worden jawel ik vind hem hartstikke mooi ze draait zich om en loopt langzaam de zoldertrap op de traptreden kraken onder haar blote voeten het is een naar geluid dat ze nog nooit eerder heeft gehoord alles klinkt een beetje anders in het donker de deur van haar slaapkamer staat op een kier het lampje boven haar bed brandt en maakt vreemde vlekken op de muur haar oude knuffelbeer staat op een kastje de vlek van de lamp op de muur erachter lijkt op een monster marte rilt en kruipt snel onder de dekens niet bang zijn het is gewoon haar eigen knuffelbeer maar haar hart bonst in haar keel voor het eerst heeft ze een eigen kamer op zolder tot gisteren sliep ze altijd met haar broer hidde op een kamer"
transcripts = ["bang in het donker", "als de beker vol is doet marte de kraam dicht", "nu moet je echt gaan slapen zegt mama", "je kwam al drie keer uit bed", "ben je niet blij met je eigen slaapkamer", "marte voelt haar wangen warm worden", "jawel ik vind hem hartstikke mooi", "ze draait zich om en loopt langzaam de zoldertramp op", "de traptreden kraken onder haar blote voeten", "het is een naar geluid dat ze nog nooit eerder heeft gehoord", "alles klinkt een beetje anders in het donker", "de deur van haar slaapkamer staat op een kier", "het lampje boven haar bed brand en maakt vreemde vlekken op de muur", "haar oude knuffelbeer staat op een kastje", "de vlek van de lamp op de mmur erachter lijkt op een monster", "marte rilt en kruipt snel onder de dekens", "niet bang zijn het is gewoon haar eigen knuffelbeer", "maar haar hart bonst in haar keel", "voor het eerst heeft ze een eigen kamer op zolder", "tot gisteren sliep ze altijd met haar broer hidde op een kamer"] #prompt.split(" ")

def align_with_transcript(
    audio : np.ndarray,
    transcripts : List[str],
    samplerate : int = SAMPLERATE,
    model : Wav2Vec2ForCTC = model,
    processor : Wav2Vec2Processor = processor,
    tokenizer : Wav2Vec2CTCTokenizer = tokenizer
):
    print('align_with_transcript')

    assert audio.ndim == 1
    # Run prediction, get logits and probabilities
    inputs = processor(audio, sampling_rate=16_000, return_tensors="pt", padding="longest")
    with torch.no_grad():
        logits = model(inputs.input_values).logits.cpu()[0]
        probs = torch.nn.functional.softmax(logits,dim=-1)
    
    # Tokenize transcripts
    vocab = tokenizer.get_vocab()
    inv_vocab = {v:k for k,v in vocab.items()}
    unk_id = vocab["[UNK]"] #vocab["<unk>"]
    
    tokens = []
    for transcript in transcripts:
        assert len(transcript) > 0
        tok_ids = tokenizer(transcript.replace("\n"," ").lower())['input_ids']
        tok_ids = np.array(tok_ids)
        tokens.append(tok_ids[tok_ids != unk_id])
    
    # Align
    char_list = [inv_vocab[i] for i in range(len(inv_vocab))]
    config = ctc_segmentation.CtcSegmentationParameters(char_list=char_list)
    config.index_duration = audio.shape[0] / probs.size()[0] / samplerate
    
    ground_truth_mat, utt_begin_indices = ctc_segmentation.prepare_token_list(config, tokens)
    timings, char_probs, state_list = ctc_segmentation.ctc_segmentation(config, probs.numpy(), ground_truth_mat)
    segments = ctc_segmentation.determine_utterance_segments(config, utt_begin_indices, char_probs, timings, transcripts)
    return [{"text" : t, "start" : p[0], "end" : p[1], "conf" : p[2]} for t,p in zip(transcripts, segments)]
    
def get_word_timestamps(
    audio : np.ndarray,
    samplerate : int = SAMPLERATE,
    model : Wav2Vec2ForCTC = model,
    processor : Wav2Vec2Processor = processor,
    tokenizer : Wav2Vec2CTCTokenizer = tokenizer
):
    print('get_word_timestamps')
    assert audio.ndim == 1
    # Run prediction, get logits and probabilities
    inputs = processor(audio, sampling_rate=16_000, return_tensors="pt", padding="longest")
    with torch.no_grad():
        logits = model(inputs.input_values).logits.cpu()[0]
        probs = torch.nn.functional.softmax(logits,dim=-1)
        
    predicted_ids = torch.argmax(logits, dim=-1)
    pred_transcript = processor.decode(predicted_ids)
    
    # Split the transcription into words
    words = pred_transcript.split(" ")
    
    # Align
    vocab = tokenizer.get_vocab()
    inv_vocab = {v:k for k,v in vocab.items()}
    char_list = [inv_vocab[i] for i in range(len(inv_vocab))]
    config = ctc_segmentation.CtcSegmentationParameters(char_list=char_list)
    config.index_duration = audio.shape[0] / probs.size()[0] / samplerate
    
    ground_truth_mat, utt_begin_indices = ctc_segmentation.prepare_text(config, words)
    timings, char_probs, state_list = ctc_segmentation.ctc_segmentation(config, probs.numpy(), ground_truth_mat)
    segments = ctc_segmentation.determine_utterance_segments(config, utt_begin_indices, char_probs, timings, words)
    return [{"text" : w, "start" : p[0], "end" : p[1], "conf" : p[2]} for w,p in zip(words, segments)]

print(align_with_transcript(audio,transcripts))
# [{'text': 'A MAN SAID TO THE UNIVERSE', 'start': 0.08124999999999993, 'end': 2.034375, 'conf': 0.0}, 
#  {'text': 'SIR I EXIST', 'start': 2.3260775862068965, 'end': 4.078771551724138, 'conf': 0.0}]

print(get_word_timestamps(audio))
# [{'text': 'a', 'start': 0.08124999999999993, 'end': 0.5912715517241378, 'conf': 0.9999501323699951}, 
# {'text': 'man', 'start': 0.5912715517241378, 'end': 0.9219827586206896, 'conf': 0.9409108982174931}, 
# {'text': 'said', 'start': 0.9219827586206896, 'end': 1.2326508620689656, 'conf': 0.7700278702302796}, 
# {'text': 'to', 'start': 1.2326508620689656, 'end': 1.3529094827586206, 'conf': 0.5094435178226225}, 
# {'text': 'the', 'start': 1.3529094827586206, 'end': 1.4831896551724135, 'conf': 0.4580493446392211}, 
# {'text': 'universe', 'start': 1.4831896551724135, 'end': 2.034375, 'conf': 0.9285054256219009}, 
# {'text': 'sir', 'start': 2.3260775862068965, 'end': 3.036530172413793, 'conf': 0.0}, 
# {'text': 'i', 'start': 3.036530172413793, 'end': 3.347198275862069, 'conf': 0.7995760873559864}, 
# {'text': 'exist', 'start': 3.347198275862069, 'end': 4.078771551724138, 'conf': 0.0}]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


align_with_transcript


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[{'text': 'bang in het donker', 'start': 0.010001195002715915, 'end': 7.5208986420423685, 'conf': 0.0}, {'text': 'als de beker vol is doet marte de kraam dicht', 'start': 7.5208986420423685, 'end': 31.903812058663767, 'conf': 0.0}, {'text': 'nu moet je echt gaan slapen zegt mama', 'start': 31.903812058663767, 'end': 37.06436892992939, 'conf': 0.0}, {'text': 'je kwam al drie keer uit bed', 'start': 37.22450755024443, 'end': 43.60515046170559, 'conf': 0.0}, {'text': 'ben je niet blij met je eigen slaapkamer', 'start': 43.905305812058664, 'end': 52.026156653992395, 'conf': 0.0}, {'text': 'marte voelt haar wangen warm worden', 'start': 55.00663226507333, 'end': 63.18749027702336, 'conf': 0.0}, {'text': 'jawel ik vind hem hartstikke mooi', 'start': 63.98770537751223, 'end': 72.22857055947854, 'conf': 0.0}, {'text': 'ze draait zich om en loopt langzaam de zoldertramp op', 'start': 73.74887170016295, 'end': 86.41032482346552, 'conf': 0.0}, {'text': 'de traptreden kraken onder haar blote voete

In [20]:
vocab = tokenizer.get_vocab()
vocab

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 'à': 27,
 'â': 28,
 'ä': 29,
 'æ': 30,
 'ç': 31,
 'è': 32,
 'é': 33,
 'ê': 34,
 'ë': 35,
 'î': 36,
 'ï': 37,
 'ó': 38,
 'ô': 39,
 'ö': 40,
 'ù': 41,
 'û': 42,
 'ü': 43,
 '|': 0,
 '[UNK]': 44,
 '[PAD]': 45,
 '<s>': 46,
 '</s>': 47}

In [ ]:
# Vocab of CGN model:
{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 'à': 27,
 'â': 28,
 'ä': 29,
 'æ': 30,
 'ç': 31,
 'è': 32,
 'é': 33,
 'ê': 34,
 'ë': 35,
 'î': 36,
 'ï': 37,
 'ó': 38,
 'ô': 39,
 'ö': 40,
 'ù': 41,
 'û': 42,
 'ü': 43,
 '|': 0,
 '[UNK]': 44,
 '[PAD]': 45,
 '<s>': 46,
 '</s>': 47}